In [4]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from ipywidgets import widgets
import plotly.express as px 
from plotly.subplots import make_subplots
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.tsa.seasonal import seasonal_decompose
import pylab
import scipy as sp
import scipy.fftpack

In [5]:
df_pp = pd.read_csv('/home/deaddemocracy/Documents/Major_Project/Extract_Feb18_28_1PM/sensor1/sensor1_pp.csv', decimal = ',',index_col = 0)

In [6]:
df_pp.head()

,ax(g),ay(g),az(g),wx(deg/s),wy(deg/s),wz(deg/s),AngleX(deg),AngleY(deg),AngleZ(deg),T(°),hx,hy,hz
ChipTime,,,,,,,,,,,,,
2020-02-18 15:40:20.960,-0.1851,0.1094,0.9927,-5.7373,-8.0566,5.188,7.3663,11.4148,2.2852,18.11,929,-1405,27
2020-02-18 15:40:21.160,-0.1533,0.0942,1.0044,-6.1646,9.5215,0.2441,6.3062,10.0964,1.7084,18.15,924,-1399,47
2020-02-18 15:40:21.360,-0.1455,0.0859,1.0083,2.4414,1.5259,-1.0986,5.9656,9.5142,1.9391,18.14,896,-1369,51
2020-02-18 15:40:21.560,-0.1299,0.1064,1.0044,3.2959,-5.0659,9.2163,6.7072,8.4705,3.7573,18.14,931,-1369,49
2020-02-18 15:40:21.760,-0.12300000000000001,0.0986,1.0044,-4.5166,-6.5918,2.6245,6.3336,7.5751,4.5593,18.14,1012,-1426,39


In [7]:
df_pp = df_pp.apply(pd.to_numeric)

In [8]:
df_pp.index

Index(['2020-02-18 15:40:20.960', '2020-02-18 15:40:21.160',
       '2020-02-18 15:40:21.360', '2020-02-18 15:40:21.560',
       '2020-02-18 15:40:21.760', '2020-02-18 15:40:21.960',
       '2020-02-18 15:40:22.160', '2020-02-18 15:40:22.360',
       '2020-02-18 15:40:22.560', '2020-02-18 15:40:22.760',
       ...
       '2020-02-28 11:41:16.960', '2020-02-28 11:41:17.160',
       '2020-02-28 11:41:17.360', '2020-02-28 11:41:17.560',
       '2020-02-28 11:41:17.760', '2020-02-28 11:41:17.960',
       '2020-02-28 11:41:18.160', '2020-02-28 11:41:18.360',
       '2020-02-28 11:41:18.560', '2020-02-28 11:41:18.760'],
      dtype='object', name='ChipTime', length=4248290)

In [9]:
df_pp.index = pd.to_datetime(df_pp.index)
type(df_pp.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [10]:
az_sliced = df_pp['az(g)'][(df_pp.index > '2020-02-19 01:48:30.960') & (df_pp.index < '2020-02-19 01:49:30.960')]

## Filter Requirements - Butterworth Low Pass

In [100]:
Fs = 5 #Hz, Sampling rate
cutoff = 2 #Low pass filter, cutoff = 2 Hz
nyq = Fs/2 #Nyquist Frequency
order = 6

## Filter Design

In [101]:
def butter_lowpass(data, cutoff, fs, order):
    normal_cutoff = cutoff / nyq #Normalization so that highest frequency = 1
    #Filter Coefficients
    b,a = sp.butter(order, normal_cutoff, btype='low', analog=False)
    y = sp.filtfilt(b,a,data)
    return y

In [102]:
az_filtered = butter_lowpass(az_sliced.values, cutoff, fs, order)

## Plot

In [103]:
fig = go.Figure()
# fig.add_trace(go.Scatter(
#             y = data,
#             line =  dict(shape =  'spline' ),
#             name = 'signal with noise'
#             ))
fig.add_trace(go.Scatter(
            y = az_filtered,
            line =  dict(shape =  'spline' ),
            name = 'filtered signal'
            ))
fig.show()


## Moving Average Filters

In [104]:
az_filtered_series = pd.Series(az_filtered)

In [105]:
az_1s = az_filtered_series.rolling(25).mean()

In [106]:
fig = go.Figure()
# fig.add_trace(go.Scatter(
#             y = data,
#             line =  dict(shape =  'spline' ),
#             name = 'signal with noise'
#             ))
fig.add_trace(go.Scatter(
            y = az_1s,
            line =  dict(shape =  'spline' ),
            name = 'Moving Average - 1s'
            ))
fig.show()

# FFT Plot 


In [13]:
az_fft = sp.fftpack.fft(az_sliced)

In [14]:
az_fft_psd = abs(az_fft) ** 2

In [21]:
fftfreq = sp.fftpack.fftfreq(len(az_fft_psd)) # 1./5 gives frequency = 1 = 1s

In [22]:
i = fftfreq > 0

In [23]:
fig = go.Figure()
fig.add_trace(go.Scatter(
            y = az_fft_psd,
            x = fftfreq[i],
            line =  dict(shape =  'spline' ),
            name = 'FFT Plot'
            ))
fig.show()